## MongoDB - Kafka Data Streaming

The following notebook illustrates how to setup continuos streaming of the data from MongoDB via to Databricks Delta Lake

Update the **TOPIC** and **KAFKA_BROKER** with relevant details.

In [0]:
import dlt
from pyspark.sql.functions import *
from pyspark.sql.types import *

TOPIC = "tracker-events"
KAFKA_BROKER = "hostname:port1, hostname:port2";  // comma separated list of broker:host


raw_kafka_events = (spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_BROKER)  
    .option("subscribe", TOPIC)
    .option("startingOffsets", "earliest")
    .load())


@dlt.table(table_properties={"pipelines.reset.allowed":"false"})
def mdb_data():
  return raw_kafka_events